# First Script: Creating the Meta-DataFrame
In this script the author analyzes each column of the synthetic BPI_C 2019 event logs.

In [34]:
#import libraries
import json
import os
import pandas as pd
import numpy
import re

In [35]:
event_log_name = 'BPI_C_2019_2_final_synthetic' #addd the name of the event log - change for each event log
event_log_file_path = f'event_logs/{event_log_name}.csv' #string interpolation, one has to create a folder called 'event_logs' to import the event log.csv
print(event_log_file_path) # review the path

event_logs/BPI_C_2019_2_final_synthetic.csv


In [36]:
#import the event log
event_log_table = pd.read_csv(event_log_file_path, encoding='ISO-8859-1') #optionally this can be added; header = 0, encoding = 'ISO-8859-1') if encoding issues arise
event_log_table.head(20) #view the first 20 rows

,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,case Vendor,Type of case Item,case Item Category,...,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event activity,event Net Worth (CHF),timestamp
0,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,vendorID_0027,Standard,"Three-way match, invoice before GR",...,vendor_0027,0,2,2000013534_00002,1,batch_00,batch_00,SRM: Document Completed,179.0,"Thu, 01 Jan 1970 00:00:10 GMT"
1,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,vendorID_0027,Standard,"Three-way match, invoice before GR",...,vendor_0027,0,2,2000013534_00002,1,batch_00,batch_00,SRM: Complete,179.0,"Thu, 01 Jan 1970 00:00:10 GMT"
2,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Laboratory Supplies & Services,2000013533,Purchase order,vendorID_0028,Standard,"Three-way match, invoice after GR",...,vendor_0028,1,2,2000013533_00002,1,batch_00,batch_00,SRM: Awaiting Approval,273.0,"Thu, 01 Jan 1970 00:00:10 GMT"
3,3.140000e+14,Additives,Company_0,Standard Purchase Order,Surfactants,4507021815,Purchase order,vendorID_0166,Standard,"Three-way match, invoice before GR",...,vendor_0164,0,10,4507021815_00010,1,user_147,user_147,Create Purchase Order Item,1416.0,"Thu, 01 Jan 1970 00:00:10 GMT"
4,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Facility Management,2000013535,Purchase order,vendorID_0000,Standard,"Three-way match, invoice before GR",...,vendor_0000,0,2,2000013535_00002,1,batch_00,batch_00,SRM: Document Completed,9.0,"Thu, 01 Jan 1970 00:00:10 GMT"
5,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Facility Management,2000013535,Purchase order,vendorID_0000,Standard,"Three-way match, invoice before GR",...,vendor_0000,0,2,2000013535_00002,1,batch_00,batch_00,SRM: Awaiting Approval,9.0,"Thu, 01 Jan 1970 00:00:10 GMT"
6,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Facility Management,2000013535,Purchase order,vendorID_0000,Standard,"Three-way match, invoice before GR",...,vendor_0000,0,2,2000013535_00002,1,batch_00,batch_00,SRM: Complete,9.0,"Thu, 01 Jan 1970 00:00:10 GMT"
7,3.140000e+14,Additives,Company_0,Standard Purchase Order,Rheology & Thixotropic Agents,4507021815,Purchase order,vendorID_0166,Standard,"Three-way match, invoice before GR",...,vendor_0164,0,20,4507021815_00020,1,user_147,user_147,Create Purchase Order Item,3073.0,"Thu, 01 Jan 1970 00:00:10 GMT"
8,3.140000e+14,Additives,Company_0,Standard Purchase Order,Rheology & Thixotropic Agents,4507021815,Purchase order,vendorID_0166,Standard,"Three-way match, invoice after GR",...,vendor_0164,1,30,4507021815_00030,1,user_147,user_147,Create Purchase Order Item,4330.0,"Thu, 01 Jan 1970 00:00:10 GMT"
9,3.010000e+14,Sales Type,Company_0,Standard Purchase Order,Products for Resale,4507021087,Purchase order,vendorID_0115,Standard,"Three-way match, invoice before GR",...,vendor_0115,0,30,4507021087_00030,1,user_019,user_019,Record Invoice Receipt,823.0,"Thu, 01 Jan 1970 00:00:10 GMT"


In [37]:
# Modification 1: Add a caseID column, that is defined as a merge of two other columns based on the requirements from the TFPM webpage
'''
  The case ID is a combination of the purchase document and the purchase item.
  https://icpmconference.org/2019/icpm-2019/contests-challenges/bpi-challenge-2019/
'''
#create the caseID and add it to the event log table
event_log_table['caseID'] = event_log_table['C_Purchasing Document'].astype('string') + ' ' + event_log_table['case Item'].astype('string')

In [38]:
#review changes on the last column
event_log_table

,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,case Vendor,Type of case Item,case Item Category,...,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event activity,event Net Worth (CHF),timestamp,caseID
0,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,vendorID_0027,Standard,"Three-way match, invoice before GR",...,0,2,2000013534_00002,1,batch_00,batch_00,SRM: Document Completed,179.0,"Thu, 01 Jan 1970 00:00:10 GMT",2000013534 2
1,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,vendorID_0027,Standard,"Three-way match, invoice before GR",...,0,2,2000013534_00002,1,batch_00,batch_00,SRM: Complete,179.0,"Thu, 01 Jan 1970 00:00:10 GMT",2000013534 2
2,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Laboratory Supplies & Services,2000013533,Purchase order,vendorID_0028,Standard,"Three-way match, invoice after GR",...,1,2,2000013533_00002,1,batch_00,batch_00,SRM: Awaiting Approval,273.0,"Thu, 01 Jan 1970 00:00:10 GMT",2000013533 2
3,3.140000e+14,Additives,Company_0,Standard Purchase Order,Surfactants,4507021815,Purchase order,vendorID_0166,Standard,"Three-way match, invoice before GR",...,0,10,4507021815_00010,1,user_147,user_147,Create Purchase Order Item,1416.0,"Thu, 01 Jan 1970 00:00:10 GMT",4507021815 10
4,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Facility Management,2000013535,Purchase order,vendorID_0000,Standard,"Three-way match, invoice before GR",...,0,2,2000013535_00002,1,batch_00,batch_00,SRM: Document Completed,9.0,"Thu, 01 Jan 1970 00:00:10 GMT",2000013535 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349520,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,vendorID_0541,Service,"Three-way match, invoice after GR",...,1,1,4507038753_00001,1,NONE,NONE,Record Service Entry Sheet,1829.0,"Thu, 01 Jan 1970 00:00:25 GMT",4507038753 1
349521,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,vendorID_0541,Service,"Three-way match, invoice after GR",...,1,1,4507038753_00001,1,NONE,NONE,Record Service Entry Sheet,3659.0,"Thu, 01 Jan 1970 00:00:25 GMT",4507038753 1
349522,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,vendorID_0541,Service,"Three-way match, invoice after GR",...,1,1,4507038753_00001,1,user_200,user_200,Record Goods Receipt,411718.0,"Thu, 01 Jan 1970 00:00:25 GMT",4507038753 1
349523,5.550000e+14,Packaging,Company_0,Standard Purchase Order,Packaging - Other,4507038569,Purchase order,vendorID_0104,Standard,"Three-way match, invoice before GR",...,0,420,4507038569_00420,1,user_063,user_063,Receive Order Confirmation,1173.0,"Thu, 01 Jan 1970 00:00:25 GMT",4507038569 420


In [39]:
#Review all og the event log columns, to decide which ones are mandatory and which ones are additional

log_columns = event_log_table.columns
log_columns # this variable is important for the Meta-DataFrame generation

Index(['event_ID ', 'case Spend type', 'Company by Case', 'C_Document Type',
       'Case Sub spend  text', 'C_Purchasing Document',
       'case Purch. Doc. Category name', 'case Vendor', 'Type of case Item ',
       'case Item Category', 'case Spend classification text', 'case Source',
       'case Name', 'CASE GR', 'case Item', 'case concept:name',
       'case Goods Receipt', 'User', 'Resource', 'event activity',
       'event Net Worth (CHF)', 'timestamp', 'caseID'],
      dtype='object')

In [40]:
# This has to be tailored to each event log
#caseID is created from 'C_Purchasing Document' and 'case Item', define the mandatory columns based on domain knowledge analysis

mandatory_columns = ['caseID', 'timestamp', 'event activity', 'event_ID ']

#Create an object/dictionary and corresponding key:value pairs to add the semantic titles which the csv titles represent in a variable  which are relevant for the Meta-DataFrame
titles = {
  'caseID': 'Case ID',
  'event activity': 'Activity',
  'event_ID ': 'Event ID',
  'timestamp': 'Timestamp',
}

print(mandatory_columns)

['caseID', 'timestamp', 'event activity', 'event_ID ']


In [41]:
#Review titles object which are relevant for the Meta-DataFrame
titles

{'caseID': 'Case ID',
 'event activity': 'Activity',
 'event_ID ': 'Event ID',
 'timestamp': 'Timestamp'}

In [42]:
#Review the datatypes for datatype harmonization
event_log_table.dtypes

event_ID                          float64
case Spend type                    object
Company by Case                    object
C_Document Type                    object
Case Sub spend  text               object
C_Purchasing Document               int64
case Purch. Doc. Category name     object
case Vendor                        object
Type of case Item                  object
case Item Category                 object
case Spend classification text     object
case Source                        object
case Name                          object
CASE GR                             int64
case Item                           int64
case concept:name                  object
case Goods Receipt                  int64
User                               object
Resource                           object
event activity                     object
event Net Worth (CHF)             float64
timestamp                          object
caseID                             string
dtype: object

In [43]:
#This step is optional to minimize the work needed in harmonization

#Translate object data types of timestamp data into a datetime64[ns] data type for the timestamp. This is a unique datatype from pandas.
timestamp_objects = event_log_table['timestamp'] #declare timestamp to a variable
event_log_table['timestamp']=pd.to_datetime(timestamp_objects)

#Review changes made
#event_log_table['timestamp'].dtypes
event_log_table.dtypes

event_ID                                      float64
case Spend type                                object
Company by Case                                object
C_Document Type                                object
Case Sub spend  text                           object
C_Purchasing Document                           int64
case Purch. Doc. Category name                 object
case Vendor                                    object
Type of case Item                              object
case Item Category                             object
case Spend classification text                 object
case Source                                    object
case Name                                      object
CASE GR                                         int64
case Item                                       int64
case concept:name                              object
case Goods Receipt                              int64
User                                           object
Resource                    

In [44]:
#This step is optional to minimize the work needed in harmonization 

#Translate object data types of object data types (which contains different possible formats) to a clear string data type format
#for loops where used as they enable somewhat automated transformations 
# = for assigning values
# == operator is used to test equality
for log_column in log_columns:
    if event_log_table[log_column].dtype == 'object': #check if the column has an object data type
        event_log_table[log_column] = event_log_table[log_column].astype('string') #change it to a string

#Review changes
#event_log_table['timestamp'].dtypes
event_log_table.dtypes


event_ID                                      float64
case Spend type                                string
Company by Case                                string
C_Document Type                                string
Case Sub spend  text                           string
C_Purchasing Document                           int64
case Purch. Doc. Category name                 string
case Vendor                                    string
Type of case Item                              string
case Item Category                             string
case Spend classification text                 string
case Source                                    string
case Name                                      string
CASE GR                                         int64
case Item                                       int64
case concept:name                              string
case Goods Receipt                              int64
User                                           string
Resource                    

In [45]:
# Reorder the columns of the event log table.
# Checking the log columns list contains each item on the mandatory columns list with the assert statement raises an 'AssertionError' if any of the elements are missing. 
# With the delete method and the == operator, the items are removed in the mandatory columns list from the log columns list.
# Then  using the + operator to combine the mandatory columns list and the log columns list (which has been turned into a list using the tolist method). By doing so, a fresh list of columns in # the desired order is created. Using the [] operator to select only the columns in the newly created list, subsetting the event log table DataFrame. By doing this, the columns in the new #DataFrame are created in the desired order.

for mandatory_column in mandatory_columns:
  assert mandatory_column in log_columns 
  log_columns = log_columns.delete(log_columns == mandatory_column)
log_columns = mandatory_columns + log_columns.tolist()

event_log_table = event_log_table[log_columns]
event_log_table #the for loop above resultes in a move of the mandatory columns to the left side (changes order)

,caseID,timestamp,event activity,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
0,2000013534 2,1970-01-01 00:00:10+00:00,SRM: Document Completed,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,...,OTHER,Source_SystemID_001,vendor_0027,0,2,2000013534_00002,1,batch_00,batch_00,179.0
1,2000013534 2,1970-01-01 00:00:10+00:00,SRM: Complete,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,...,OTHER,Source_SystemID_001,vendor_0027,0,2,2000013534_00002,1,batch_00,batch_00,179.0
2,2000013533 2,1970-01-01 00:00:10+00:00,SRM: Awaiting Approval,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Laboratory Supplies & Services,2000013533,Purchase order,...,NPR,Source_SystemID_001,vendor_0028,1,2,2000013533_00002,1,batch_00,batch_00,273.0
3,4507021815 10,1970-01-01 00:00:10+00:00,Create Purchase Order Item,3.140000e+14,Additives,Company_0,Standard Purchase Order,Surfactants,4507021815,Purchase order,...,PR,Source_SystemID_001,vendor_0164,0,10,4507021815_00010,1,user_147,user_147,1416.0
4,2000013535 2,1970-01-01 00:00:10+00:00,SRM: Document Completed,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Facility Management,2000013535,Purchase order,...,NPR,Source_SystemID_001,vendor_0000,0,2,2000013535_00002,1,batch_00,batch_00,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349520,4507038753 1,1970-01-01 00:00:25+00:00,Record Service Entry Sheet,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,...,NPR,Source_SystemID_001,vendor_0525,1,1,4507038753_00001,1,NONE,NONE,1829.0
349521,4507038753 1,1970-01-01 00:00:25+00:00,Record Service Entry Sheet,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,...,NPR,Source_SystemID_001,vendor_0525,1,1,4507038753_00001,1,NONE,NONE,3659.0
349522,4507038753 1,1970-01-01 00:00:25+00:00,Record Goods Receipt,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,...,NPR,Source_SystemID_001,vendor_0525,1,1,4507038753_00001,1,user_200,user_200,411718.0
349523,4507038569 420,1970-01-01 00:00:25+00:00,Receive Order Confirmation,5.550000e+14,Packaging,Company_0,Standard Purchase Order,Packaging - Other,4507038569,Purchase order,...,PR,Source_SystemID_001,vendor_0104,0,420,4507038569_00420,1,user_063,user_063,1173.0


Here we are going to reset the index and reorder the event log with a new index

Setting the inplace parameter to True and using the reset index method to reset the DataFrame's index. 
This adds a new column to the DataFrame containing the old index values and resets the index to a range of sequential integers beginning at 0. 
Using the rename method and the inplace parameter set to True, rename the column containing the previous index values. 
A dictionary that maps the old column name "index" to the new column name "EventIndex" makes up the columns parameter.
Using the set index method and the inplace parameter set to True, the DataFrame's index is changed to the 'EventIndex' column. 
This will remove the "EventIndex" column from the data and make it the new index of the DataFrame.

In [46]:
# Number every single event in the event log.
# Every single event is represented by a line in the event log file.
event_log_table.reset_index(inplace=True)
event_log_table.rename(columns={'index': 'EventIndex'}, inplace=True)
event_log_table.set_index('EventIndex', inplace=True)

#review changes
event_log_table

/var/folders/vz/snd73j8d2mj817493bt9ykfc0000gn/T/ipykernel_98633/2322881527.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_log_table.rename(columns={'index': 'EventIndex'}, inplace=True)


,caseID,timestamp,event activity,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
EventIndex,,,,,,,,,,,,,,,,,,,,,
0,2000013534 2,1970-01-01 00:00:10+00:00,SRM: Document Completed,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,...,OTHER,Source_SystemID_001,vendor_0027,0,2,2000013534_00002,1,batch_00,batch_00,179.0
1,2000013534 2,1970-01-01 00:00:10+00:00,SRM: Complete,1.890000e+14,Others,Company_0,PO EC,Raw Material,2000013534,Purchase order,...,OTHER,Source_SystemID_001,vendor_0027,0,2,2000013534_00002,1,batch_00,batch_00,179.0
2,2000013533 2,1970-01-01 00:00:10+00:00,SRM: Awaiting Approval,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Laboratory Supplies & Services,2000013533,Purchase order,...,NPR,Source_SystemID_001,vendor_0028,1,2,2000013533_00002,1,batch_00,batch_00,273.0
3,4507021815 10,1970-01-01 00:00:10+00:00,Create Purchase Order Item,3.140000e+14,Additives,Company_0,Standard Purchase Order,Surfactants,4507021815,Purchase order,...,PR,Source_SystemID_001,vendor_0164,0,10,4507021815_00010,1,user_147,user_147,1416.0
4,2000013535 2,1970-01-01 00:00:10+00:00,SRM: Document Completed,1.890000e+14,CAPEX & SOCS,Company_0,PO EC,Facility Management,2000013535,Purchase order,...,NPR,Source_SystemID_001,vendor_0000,0,2,2000013535_00002,1,batch_00,batch_00,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349520,4507038753 1,1970-01-01 00:00:25+00:00,Record Service Entry Sheet,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,...,NPR,Source_SystemID_001,vendor_0525,1,1,4507038753_00001,1,NONE,NONE,1829.0
349521,4507038753 1,1970-01-01 00:00:25+00:00,Record Service Entry Sheet,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,...,NPR,Source_SystemID_001,vendor_0525,1,1,4507038753_00001,1,NONE,NONE,3659.0
349522,4507038753 1,1970-01-01 00:00:25+00:00,Record Goods Receipt,5.580000e+14,Logistics,Company_0,Standard Purchase Order,Road Packed,4507038753,Purchase order,...,NPR,Source_SystemID_001,vendor_0525,1,1,4507038753_00001,1,user_200,user_200,411718.0


In [47]:
#defining a new dictionary  overview_columns with log columns as the elements
content_overview_columns = [(log_column, ['BPI_C_2019_2nd_Supplier']) for log_column in log_columns]
extra_overview_columns = [('Event log column property', ['BPI_C_2019_2nd_Supplier'])]
overview_columns = dict(extra_overview_columns + content_overview_columns)
overview_table = pd.DataFrame(overview_columns)

# To add a vertically indented name to the index in the console output.
overview_table.set_index('Event log column property', inplace=True)
overview_columns

{'Event log column property': ['BPI_C_2019_2nd_Supplier'],
 'caseID': ['BPI_C_2019_2nd_Supplier'],
 'timestamp': ['BPI_C_2019_2nd_Supplier'],
 'event activity': ['BPI_C_2019_2nd_Supplier'],
 'event_ID ': ['BPI_C_2019_2nd_Supplier'],
 'case Spend type': ['BPI_C_2019_2nd_Supplier'],
 'Company by Case': ['BPI_C_2019_2nd_Supplier'],
 'C_Document Type': ['BPI_C_2019_2nd_Supplier'],
 'Case Sub spend  text': ['BPI_C_2019_2nd_Supplier'],
 'C_Purchasing Document': ['BPI_C_2019_2nd_Supplier'],
 'case Purch. Doc. Category name': ['BPI_C_2019_2nd_Supplier'],
 'case Vendor': ['BPI_C_2019_2nd_Supplier'],
 'Type of case Item ': ['BPI_C_2019_2nd_Supplier'],
 'case Item Category': ['BPI_C_2019_2nd_Supplier'],
 'case Spend classification text': ['BPI_C_2019_2nd_Supplier'],
 'case Source': ['BPI_C_2019_2nd_Supplier'],
 'case Name': ['BPI_C_2019_2nd_Supplier'],
 'CASE GR': ['BPI_C_2019_2nd_Supplier'],
 'case Item': ['BPI_C_2019_2nd_Supplier'],
 'case concept:name': ['BPI_C_2019_2nd_Supplier'],
 'case Good

In [48]:
#view the new overview_table to construct the Meta-DataFrame
overview_table 

,caseID,timestamp,event activity,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
Event log column property,,,,,,,,,,,,,,,,,,,,,
BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,...,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier


<h2>Multiple variables based on event data proprocessing which are used for the Meta-DataFrame</h2>

The author uses the pandas .loc function which is an indexer in Pandas is used to choose rows and columns from a DataFrame based on labels.

The snippet below is called title property X as it will be added to the overview table (Meta-DataFrame) with the log columns list's elements as keys and the log column itself as a value, if the title associated with the log column is present in the titles dictionary (if it does not exist in the titles dictionary). 

Then, a new row with the label "Title" and the values from the title property dictionary is added to the overview table DataFrame. The assignment will add the new row to the DataFrame, and the.loc indexer will be used to select the row by its label. 

When this code block is run, a new row with the label "Title" and values representing the names of each log column will appear in the overview table DataFrame (or the log column itself if no title is provided).

In [49]:
# Overview for property: Titles - For the Meta-DataFrame
# Depending on whether the title for the log column is present in the titles dictionary, the values of the log columns list are either the log column itself or its associated title (if it does not exist in the titles dictionary).
title_property = {}

for log_column in log_columns:
    if log_column in titles:
        title = titles[log_column]
    else:
        title = log_column
    title_property[log_column] = title


Then, a new row with the label "Title" and the values from the title property dictionary is added to the overview table DataFrame. The assignment will add the new row to the DataFrame, and the.loc indexer will be used to select the row by its label. 
When this code block is run, a new row with the label "Title" and values representing the names of each log column will appear in the overview table DataFrame (or the log column itself if no title is provided).

In [50]:
overview_table.loc['Title'] = title_property
overview_table.loc['Title']

caseID                                                   Case ID
timestamp                                              Timestamp
event activity                                          Activity
event_ID                                                Event ID
case Spend type                                  case Spend type
Company by Case                                  Company by Case
C_Document Type                                  C_Document Type
Case Sub spend  text                        Case Sub spend  text
C_Purchasing Document                      C_Purchasing Document
case Purch. Doc. Category name    case Purch. Doc. Category name
case Vendor                                          case Vendor
Type of case Item                             Type of case Item 
case Item Category                            case Item Category
case Spend classification text    case Spend classification text
case Source                                          case Source
case Name                

<h2>Dictionary {for mandatory and additional attributes}</h2>

This code block generates a new dictionary called importance property whose keys are the items in the list of log columns and whose values are either "Mandatory attribute" or "Additional attribute" depending on whether the log column is included in the mandatory columns list or not. 
Then, a new row with the label "Importance" and values taken from the importance property dictionary is added to the overview table DataFrame. The assignment will add the new row to the DataFrame, and the.loc indexer will be used to select the row by its label. 
When this code block is run, a new row with the label "Importance" and the values "Mandatory attribute" or "Additional attribute" for each log column, depending on whether that column is in the mandatory columns list or not, will appear in the overview table DataFrame.

The concept is used for other data attributes as well

In [51]:
# Overview property: Label for columns either 'Mandatory attribute' or 'additional_attributes' - For the Meta-DataFrame

importance_property = {}
for log_column in log_columns:
    if log_column in mandatory_columns:
        importance_property[log_column] = 'Mandatory attribute'
    else:
        importance_property[log_column] = 'Additional attribute'

overview_table.loc['Importance'] = importance_property
overview_table.loc['Importance']


caseID                             Mandatory attribute
timestamp                          Mandatory attribute
event activity                     Mandatory attribute
event_ID                           Mandatory attribute
case Spend type                   Additional attribute
Company by Case                   Additional attribute
C_Document Type                   Additional attribute
Case Sub spend  text              Additional attribute
C_Purchasing Document             Additional attribute
case Purch. Doc. Category name    Additional attribute
case Vendor                       Additional attribute
Type of case Item                 Additional attribute
case Item Category                Additional attribute
case Spend classification text    Additional attribute
case Source                       Additional attribute
case Name                         Additional attribute
CASE GR                           Additional attribute
case Item                         Additional attribute
case conce

The goal of this code snippet is to add a new row to the Meta-Dataframe termed overview table, where the keys are the columns in a list called log columns and the values are strings indicating whether the column is at the "Event level" or the "Case level." The values are determined by examining the column name for the string "event" or "case," respectively.

In [52]:
# Overview property: The level of the event data attribute in the overview table
level_property = {}
for log_column in log_columns:
  if 'event' in log_column:
    level_property[log_column] = 'Event level'
  if 'case' in log_column:
    level_property[log_column] = 'Case level'

overview_table.loc['Level'] = level_property
overview_table.loc['Level']

#NaN's can manually be changed in the Meta-DataFrame

caseID                             Case level
timestamp                                 NaN
event activity                    Event level
event_ID                          Event level
case Spend type                    Case level
Company by Case                           NaN
C_Document Type                           NaN
Case Sub spend  text                      NaN
C_Purchasing Document                     NaN
case Purch. Doc. Category name     Case level
case Vendor                        Case level
Type of case Item                  Case level
case Item Category                 Case level
case Spend classification text     Case level
case Source                        Case level
case Name                          Case level
CASE GR                                   NaN
case Item                          Case level
case concept:name                  Case level
case Goods Receipt                 Case level
User                                      NaN
Resource                          

In [53]:
# Overview property: Data type
data_type_property = {}
datatypes = event_log_table.dtypes
for log_column in log_columns:
  data_type_property[log_column] = datatypes[log_column]

overview_table.loc['Data type'] = data_type_property

This snippet adds a new row to the Meta-Dataframe in Pandas, with the keys being the columns in a list called log columns and the values being the data types of the corresponding columns in an event log table DataFrame.

In [54]:
# Overview property: Data type
data_type_property = {}
datatypes = event_log_table.dtypes
for log_column in log_columns:
    data_type_property[log_column] = datatypes[log_column]

overview_table.loc['Data type'] = data_type_property
overview_table.loc['Data type']

caseID                                         string
timestamp                         datetime64[ns, UTC]
event activity                                 string
event_ID                                      float64
case Spend type                                string
Company by Case                                string
C_Document Type                                string
Case Sub spend  text                           string
C_Purchasing Document                           int64
case Purch. Doc. Category name                 string
case Vendor                                    string
Type of case Item                              string
case Item Category                             string
case Spend classification text                 string
case Source                                    string
case Name                                      string
CASE GR                                         int64
case Item                                       int64
case concept:name           

In [55]:
# Overview property: Count the number of entries for the Meta-Dataframe
number_of_entries_property = {}
for log_column in log_columns:
    number_of_entries_property[log_column] = len(event_log_table)

overview_table.loc['Number of entries'] = number_of_entries_property
overview_table.loc['Number of entries']

caseID                            349525
timestamp                         349525
event activity                    349525
event_ID                          349525
case Spend type                   349525
Company by Case                   349525
C_Document Type                   349525
Case Sub spend  text              349525
C_Purchasing Document             349525
case Purch. Doc. Category name    349525
case Vendor                       349525
Type of case Item                 349525
case Item Category                349525
case Spend classification text    349525
case Source                       349525
case Name                         349525
CASE GR                           349525
case Item                         349525
case concept:name                 349525
case Goods Receipt                349525
User                              349525
Resource                          349525
event Net Worth (CHF)             349525
Name: Number of entries, dtype: object

In [56]:
# Overview property: Number of unique entries
number_of_unique_entries_property = {}
for log_column in log_columns:
  count = event_log_table[log_column].value_counts().count()
  number_of_unique_entries_property[log_column] = count

overview_table.loc['Number of unique entries'] = number_of_unique_entries_property
overview_table.loc['Number of unique entries']

caseID                            100632
timestamp                             31
event activity                        37
event_ID                            1804
case Spend type                       20
Company by Case                        3
C_Document Type                        3
Case Sub spend  text                 131
C_Purchasing Document              30584
case Purch. Doc. Category name         1
case Vendor                         1486
Type of case Item                      6
case Item Category                     4
case Spend classification text         3
case Source                            1
case Name                           1420
CASE GR                                2
case Item                            317
case concept:name                 100632
case Goods Receipt                     2
User                                 339
Resource                             339
event Net Worth (CHF)              14542
Name: Number of unique entries, dtype: object

In [57]:
#Overview property: List of up to 100 unique entries (to grasp an understanding of how the data looks and it is structured and not just metrics)
unique_entries_property = {}
for log_column in log_columns:
    unique_entries = event_log_table[log_column].unique()
    unique_entries = list(unique_entries)
    unique_entries = unique_entries[:100]
    unique_entries_property[log_column] = unique_entries

overview_table.loc['List up to 100 unique entries to view the structure of the event attributes']= unique_entries_property
unique_entries_property

{'caseID': ['2000013534 2',
  '2000013533 2',
  '4507021815 10',
  '2000013535 2',
  '4507021815 20',
  '4507021815 30',
  '4507021087 30',
  '2000013534 1',
  '4507021087 20',
  '2000013533 1',
  '2000013535 1',
  '4507021742 10',
  '4507021742 20',
  '4507021087 40',
  '4507013238 10',
  '4507021816 10',
  '4507021817 10',
  '4507021817 20',
  '4507021818 10',
  '4507021819 10',
  '4507021819 20',
  '4507021819 30',
  '4507021819 40',
  '4507021820 10',
  '4507021820 20',
  '4507020509 60',
  '4507021822 20',
  '4507021823 10',
  '4507021824 10',
  '4507021821 10',
  '4507007376 20',
  '4507021822 10',
  '4507021825 10',
  '4507019873 10',
  '4507019873 20',
  '4507018514 10',
  '4507019873 30',
  '4507019920 10',
  '4507019873 40',
  '4507019920 20',
  '4507020509 180',
  '4507021378 50',
  '4507021378 40',
  '4507019873 50',
  '4507021378 30',
  '4507021378 20',
  '4507021378 10',
  '4507020340 130',
  '4507021826 10',
  '4507017373 20',
  '4507017373 10',
  '4507020509 130',
  '45

In [58]:
# Overview property: Length of the list of up to 100 unique entries
num_displayed_unique_entries_property = {}
for log_column in log_columns:
  unique_entries = event_log_table[log_column].unique()
  unique_entries = list(unique_entries)
  unique_entries = unique_entries[:100]
  num_displayed_unique_entries_property[log_column] = len(unique_entries)

overview_table.loc['Length of the list of up to 100 unique entries'] = num_displayed_unique_entries_property
num_displayed_unique_entries_property

{'caseID': 100,
 'timestamp': 31,
 'event activity': 37,
 'event_ID ': 100,
 'case Spend type': 21,
 'Company by Case': 3,
 'C_Document Type': 3,
 'Case Sub spend  text': 100,
 'C_Purchasing Document': 100,
 'case Purch. Doc. Category name': 1,
 'case Vendor': 100,
 'Type of case Item ': 6,
 'case Item Category': 4,
 'case Spend classification text': 4,
 'case Source': 1,
 'case Name': 100,
 'CASE GR': 2,
 'case Item': 100,
 'case concept:name': 100,
 'case Goods Receipt': 2,
 'User': 100,
 'Resource': 100,
 'event Net Worth (CHF)': 100}

In [59]:
# Inspect the overview table
overview_table

,caseID,timestamp,event activity,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
Event log column property,,,,,,,,,,,,,,,,,,,,,
BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,...,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier
Title,Case ID,Timestamp,Activity,Event ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
Importance,Mandatory attribute,Mandatory attribute,Mandatory attribute,Mandatory attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,...,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute
Level,Case level,NaN,Event level,Event level,Case level,NaN,NaN,NaN,NaN,Case level,...,Case level,Case level,Case level,NaN,Case level,Case level,Case level,NaN,NaN,Event level
Data type,string,"datetime64[ns, UTC]",string,float64,string,string,string,string,int64,string,...,string,string,string,int64,int64,string,int64,string,string,float64
Number of entries,349525,349525,349525,349525,349525,349525,349525,349525,349525,349525,...,349525,349525,349525,349525,349525,349525,349525,349525,349525,349525
Number of unique entries,100632,31,37,1804,20,3,3,131,30584,1,...,3,1,1420,2,317,100632,2,339,339,14542
List up to 100 unique entries to view the structure of the event attributes,"[2000013534 2, 2000013533 2, 4507021815 10, 20...","[1970-01-01 00:00:10+00:00, 1970-01-01 00:00:1...","[SRM: Document Completed, SRM: Complete, SRM: ...","[189000000000000.0, 314000000000000.0, 3010000...","[Others, CAPEX & SOCS, Additives, Sales Type, ...","[Company_0, Company_3, Company_1]","[PO EC , Standard Purchase Order, Framework Ty...","[Raw Material, Laboratory Supplies & Services,...","[2000013534, 2000013533, 4507021815, 200001353...",[Purchase order],...,"[OTHER, NPR, PR, <NA>]",[Source_SystemID_001],"[vendor_0027, vendor_0028, vendor_0164, vendor...","[0, 1]","[2, 10, 20, 30, 1, 40, 60, 180, 50, 130, 70, 8...","[2000013534_00002, 2000013533_00002, 450702181...","[1, 0]","[batch_00, user_147, user_019, user_029, user_...","[batch_00, user_147, user_019, user_029, user_...","[179.0, 273.0, 1416.0, 9.0, 3073.0, 4330.0, 82..."
Length of the list of up to 100 unique entries,100,31,37,100,21,3,3,100,100,1,...,4,1,100,2,100,100,2,100,100,100


In [60]:
# Overview property: Number of duplicate entries
number_of_duplicate_entries_property = {}
for log_column in log_columns:
  number_of_duplicate_entries_property[log_column] = number_of_entries_property[log_column] - number_of_unique_entries_property[log_column]

overview_table.loc['Number of duplicate entries'] = number_of_duplicate_entries_property
overview_table.loc['Number of duplicate entries']

caseID                            248893
timestamp                         349494
event activity                    349488
event_ID                          347721
case Spend type                   349505
Company by Case                   349522
C_Document Type                   349522
Case Sub spend  text              349394
C_Purchasing Document             318941
case Purch. Doc. Category name    349524
case Vendor                       348039
Type of case Item                 349519
case Item Category                349521
case Spend classification text    349522
case Source                       349524
case Name                         348105
CASE GR                           349523
case Item                         349208
case concept:name                 248893
case Goods Receipt                349523
User                              349186
Resource                          349186
event Net Worth (CHF)             334983
Name: Number of duplicate entries, dtype: object

In [61]:
# Overview property: Number of undefined entries
number_of_undefined_entries_property = {}
nan_info = event_log_table.isna().sum()
for log_column in log_columns:
    number_of_undefined_entries_property[log_column] = nan_info[log_column]

overview_table.loc['Number of undefined entries'] = number_of_undefined_entries_property
number_of_undefined_entries_property

{'caseID': 0,
 'timestamp': 0,
 'event activity': 0,
 'event_ID ': 0,
 'case Spend type': 2762,
 'Company by Case': 0,
 'C_Document Type': 0,
 'Case Sub spend  text': 2762,
 'C_Purchasing Document': 0,
 'case Purch. Doc. Category name': 0,
 'case Vendor': 0,
 'Type of case Item ': 0,
 'case Item Category': 0,
 'case Spend classification text': 2762,
 'case Source': 0,
 'case Name': 0,
 'CASE GR': 0,
 'case Item': 0,
 'case concept:name': 0,
 'case Goods Receipt': 0,
 'User': 0,
 'Resource': 0,
 'event Net Worth (CHF)': 0}

In [62]:
# Overview propperty: Number of undefined entries in percentages
percentage_of_undefined_entries_property = {}
nan_info = event_log_table.isna().sum()
for log_column in log_columns:
    percentage = nan_info[log_column]/len(event_log_table)*100 #len calculates the length of a variable
    percentage = f'{percentage:.2f}'
    percentage_of_undefined_entries_property[log_column] = percentage

overview_table.loc['Percentage of undefined entries'] = percentage_of_undefined_entries_property
percentage_of_undefined_entries_property

{'caseID': '0.00',
 'timestamp': '0.00',
 'event activity': '0.00',
 'event_ID ': '0.00',
 'case Spend type': '0.79',
 'Company by Case': '0.00',
 'C_Document Type': '0.00',
 'Case Sub spend  text': '0.79',
 'C_Purchasing Document': '0.00',
 'case Purch. Doc. Category name': '0.00',
 'case Vendor': '0.00',
 'Type of case Item ': '0.00',
 'case Item Category': '0.00',
 'case Spend classification text': '0.79',
 'case Source': '0.00',
 'case Name': '0.00',
 'CASE GR': '0.00',
 'case Item': '0.00',
 'case concept:name': '0.00',
 'case Goods Receipt': '0.00',
 'User': '0.00',
 'Resource': '0.00',
 'event Net Worth (CHF)': '0.00'}

In [63]:
overview_table

,caseID,timestamp,event activity,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
Event log column property,,,,,,,,,,,,,,,,,,,,,
BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,...,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier
Title,Case ID,Timestamp,Activity,Event ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
Importance,Mandatory attribute,Mandatory attribute,Mandatory attribute,Mandatory attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,...,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute
Level,Case level,NaN,Event level,Event level,Case level,NaN,NaN,NaN,NaN,Case level,...,Case level,Case level,Case level,NaN,Case level,Case level,Case level,NaN,NaN,Event level
Data type,string,"datetime64[ns, UTC]",string,float64,string,string,string,string,int64,string,...,string,string,string,int64,int64,string,int64,string,string,float64
Number of entries,349525,349525,349525,349525,349525,349525,349525,349525,349525,349525,...,349525,349525,349525,349525,349525,349525,349525,349525,349525,349525
Number of unique entries,100632,31,37,1804,20,3,3,131,30584,1,...,3,1,1420,2,317,100632,2,339,339,14542
List up to 100 unique entries to view the structure of the event attributes,"[2000013534 2, 2000013533 2, 4507021815 10, 20...","[1970-01-01 00:00:10+00:00, 1970-01-01 00:00:1...","[SRM: Document Completed, SRM: Complete, SRM: ...","[189000000000000.0, 314000000000000.0, 3010000...","[Others, CAPEX & SOCS, Additives, Sales Type, ...","[Company_0, Company_3, Company_1]","[PO EC , Standard Purchase Order, Framework Ty...","[Raw Material, Laboratory Supplies & Services,...","[2000013534, 2000013533, 4507021815, 200001353...",[Purchase order],...,"[OTHER, NPR, PR, <NA>]",[Source_SystemID_001],"[vendor_0027, vendor_0028, vendor_0164, vendor...","[0, 1]","[2, 10, 20, 30, 1, 40, 60, 180, 50, 130, 70, 8...","[2000013534_00002, 2000013533_00002, 450702181...","[1, 0]","[batch_00, user_147, user_019, user_029, user_...","[batch_00, user_147, user_019, user_029, user_...","[179.0, 273.0, 1416.0, 9.0, 3073.0, 4330.0, 82..."
Length of the list of up to 100 unique entries,100,31,37,100,21,3,3,100,100,1,...,4,1,100,2,100,100,2,100,100,100


In [64]:
# Switching the index of the Meta-DataFrame and table structure
# The current layout of the Meta-Dataframe is transformed, that the columns become the rows, and the rows become the columns.
overview_table = overview_table.T
overview_table.reset_index(inplace = True)
overview_table.rename(columns={'index': 'Property'}, inplace = True)
overview_table

Event log column property,Property,BPI_C_2019_2nd_Supplier,Title,Importance,Level,Data type,Number of entries,Number of unique entries,List up to 100 unique entries to view the structure of the event attributes,Length of the list of up to 100 unique entries,Number of duplicate entries,Number of undefined entries,Percentage of undefined entries
0,caseID,BPI_C_2019_2nd_Supplier,Case ID,Mandatory attribute,Case level,string,349525,100632,"[2000013534 2, 2000013533 2, 4507021815 10, 20...",100,248893,0,0.00
1,timestamp,BPI_C_2019_2nd_Supplier,Timestamp,Mandatory attribute,NaN,"datetime64[ns, UTC]",349525,31,"[1970-01-01 00:00:10+00:00, 1970-01-01 00:00:1...",31,349494,0,0.00
2,event activity,BPI_C_2019_2nd_Supplier,Activity,Mandatory attribute,Event level,string,349525,37,"[SRM: Document Completed, SRM: Complete, SRM: ...",37,349488,0,0.00
3,event_ID,BPI_C_2019_2nd_Supplier,Event ID,Mandatory attribute,Event level,float64,349525,1804,"[189000000000000.0, 314000000000000.0, 3010000...",100,347721,0,0.00
4,case Spend type,BPI_C_2019_2nd_Supplier,case Spend type,Additional attribute,Case level,string,349525,20,"[Others, CAPEX & SOCS, Additives, Sales Type, ...",21,349505,2762,0.79
5,Company by Case,BPI_C_2019_2nd_Supplier,Company by Case,Additional attribute,NaN,string,349525,3,"[Company_0, Company_3, Company_1]",3,349522,0,0.00
6,C_Document Type,BPI_C_2019_2nd_Supplier,C_Document Type,Additional attribute,NaN,string,349525,3,"[PO EC , Standard Purchase Order, Framework Ty...",3,349522,0,0.00
7,Case Sub spend text,BPI_C_2019_2nd_Supplier,Case Sub spend text,Additional attribute,NaN,string,349525,131,"[Raw Material, Laboratory Supplies & Services,...",100,349394,2762,0.79
8,C_Purchasing Document,BPI_C_2019_2nd_Supplier,C_Purchasing Document,Additional attribute,NaN,int64,349525,30584,"[2000013534, 2000013533, 4507021815, 200001353...",100,318941,0,0.00
9,case Purch. Doc. Category name,BPI_C_2019_2nd_Supplier,case Purch. Doc. Category name,Additional attribute,Case level,string,349525,1,[Purchase order],1,349524,0,0.00


In [65]:
#Now we will out put this csv in the analysis folder

if not os.path.exists('analysis'):
    os.mkdir('analysis')
overview_table.to_csv(f'analysis/overview_{event_log_name}.csv')

(f'Written: datatype table to analysis/overview_{event_log_name}.csv')

'Written: datatype table to analysis/overview_BPI_C_2019_2_final_synthetic.csv'

In [66]:
#This is an exploratory code snippet checking for the longest case to model the graph in a deperate database to reveal the event activity sequences in a directly follows graph
case_sizes = event_log_table.groupby(['caseID']).size()
#use of argmax() instead of max()
index_of_longest_case = case_sizes.argmax()
caseID_of_longest_case = case_sizes.index[index_of_longest_case] 

# Without the .copy() suffix, we get: A value is trying to be set on a copy of a slice from a DataFrame
case_rows = event_log_table[event_log_table['caseID'] == caseID_of_longest_case].copy()
case_rows.sort_values(by='timestamp', ascending=True, inplace=True)
case_rows.to_csv(f'analysis/longest_case_in_{event_log_name}.csv')

# a csv table is created in the analysis file which is then transformed into Neo4j commands 

# End of Script 1 to generate the Meta-DataFrame
Author: Kyle Smith <br>
Script: For Masterthesis <br>
University: University of Camerino & University of Applied Sciences Northwestern Switzerland